# MultiRetrievalQAChain

In [1]:
from langchain.chains.router import MultiRetrievalQAChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

from langchain.embeddings import HuggingFaceInstructEmbeddings
import torch

embedding_model = HuggingFaceInstructEmbeddings(
        model_name = 'hkunlp/instructor-base',              
        model_kwargs = {
            'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        },
    )

sou_docs = TextLoader('../docs/txt/state_of_the_union.txt').load_and_split()
sou_retriever = FAISS.from_documents(sou_docs, embedding_model).as_retriever()

pg_docs = TextLoader('../docs/txt/paul_graham_essay.txt').load_and_split()
pg_retriever = FAISS.from_documents(pg_docs, embedding_model).as_retriever()

personal_texts = [
    "I love apple pie",
    "My favorite color is fuchsia",
    "My dream is to become a professional dancer",
    "I broke my arm when I was 12",
    "My parents are from Peru",
]
personal_retriever = FAISS.from_texts(personal_texts, embedding_model).as_retriever()

/Users/guntsv/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/guntsv/Library/Python/3.9/lib/python/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/guntsv/Library/Python/3.9/lib/python/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


max_seq_length  512


In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_template = "You are a nice chatbot having a conversation with a human."

default_chain = LLMChain(
    llm=OpenAI(), 
    prompt=PromptTemplate.from_template(prompt_template))

retriever_infos = [
    {
        "name": "state of the union",
        "description": "Good for answering questions about the 2023 State of the Union address",
        "retriever": sou_retriever
    },
    {
        "name": "pg essay",
        "description": "Good for answering questions about Paul Graham's essay on his career",
        "retriever": pg_retriever
    },
    {
        "name": "personal",
        "description": "Good for answering questions about me",
        "retriever": personal_retriever
    }
]

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    k = 3,  
    memory_key = "chat_history", 
    return_messages = True,  
    output_key = 'result')

# MultiRetrievalQAChain doesn't return_source_documents therefore it have to edit library. 
# chain = RetrievalQA.from_llm(llm, prompt=prompt, retriever=retriever, return_source_documents=True)
# _default_chain = RetrievalQA.from_llm(llm, prompt=default_prompt, retriever=default_retriever, return_source_documents=True)
chain = MultiRetrievalQAChain.from_retrievers(
    llm = OpenAI(), 
    retriever_infos= retriever_infos,  
    default_chain= default_chain,
    verbose=True,
    memory=memory)

In [3]:
answer = chain({"input": "What about my background"})
answer



> Entering new MultiRetrievalQAChain chain...


/Users/guntsv/Library/Python/3.9/lib/python/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


personal: {'query': 'What about my background'}
> Finished chain.


{'input': 'What about my background',
 'chat_history': [],
 'query': 'What about my background',
 'result': ' Your background includes your parents from Peru, breaking your arm when you were 12, your favorite color being fuchsia, and your dream of becoming a professional dancer.',
 'source_documents': [Document(page_content='My parents are from Peru'),
  Document(page_content='I broke my arm when I was 12'),
  Document(page_content='My favorite color is fuchsia'),
  Document(page_content='My dream is to become a professional dancer')]}

In [4]:
answer = chain({"input": "What about my parent"})
answer



> Entering new MultiRetrievalQAChain chain...


/Users/guntsv/Library/Python/3.9/lib/python/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


personal: {'query': 'What about my parents?'}

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-Ds6D2AMs3kLcrGzbiTbp1Ifs on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-Ds6D2AMs3kLcrGzbiTbp1Ifs on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals


> Finished chain.


{'input': 'What about my parent',
 'chat_history': [HumanMessage(content='What about my background'),
  AIMessage(content=' Your background includes your parents from Peru, breaking your arm when you were 12, your favorite color being fuchsia, and your dream of becoming a professional dancer.')],
 'query': 'What about my parents?',
 'result': ' Your parents are from Peru.',
 'source_documents': [Document(page_content='My parents are from Peru'),
  Document(page_content='I broke my arm when I was 12'),
  Document(page_content='My favorite color is fuchsia'),
  Document(page_content='My dream is to become a professional dancer')]}